# collaborative recommendation system

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
ratings = pd.read_csv("r.csv")
movies = pd.read_csv("m.csv")

Mean = ratings.groupby(by="userId",as_index=False)['rating'].mean()
Rating_avg = pd.merge(ratings,Mean,on='userId')
Rating_avg['adg_rating']=Rating_avg['rating_x']-Rating_avg['rating_y']
Rating_avg.head()

,userId,movieId,rating_x,timestamp,rating_y,adg_rating
0,1,1,4.0,964982703,4.366379,-0.366379
1,1,3,4.0,964981247,4.366379,-0.366379
2,1,6,4.0,964982224,4.366379,-0.366379
3,1,47,5.0,964983815,4.366379,0.633621
4,1,50,5.0,964982931,4.366379,0.633621


In [3]:
ratings = pd.merge(movies,ratings).drop(['genres','timestamp'],axis=1)
ratings.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [4]:
user_ratings = ratings.pivot_table(index=['title'],columns=['userId'],values='rating')
user_ratings

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
'Hellboy': The Seeds of Creation (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Round Midnight (1986),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Salem's Lot (2004),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Til There Was You (1997),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ (1999),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,NaN,NaN,NaN,NaN,4.5,NaN,NaN
xXx (2002),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,2.0
xXx: State of the Union (2005),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5


In [5]:
user_ratings = user_ratings.dropna(thresh=10,axis=1).fillna(0)
user_ratings

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Til There Was You (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0
xXx (2002),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,2.0
xXx: State of the Union (2005),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5


In [6]:
item_similarity_df=user_ratings.corr(method='pearson')
item_similarity_df

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.019396,0.053052,0.176911,0.120862,0.104406,0.143785,0.128542,0.055263,-0.000307,...,0.066248,0.149934,0.186959,0.056523,0.134402,0.121958,0.254192,0.262225,0.085430,0.098693
2,0.019396,1.000000,-0.002595,-0.003808,0.013181,0.016252,0.021564,0.023748,-0.003450,0.061877,...,0.198547,0.010885,-0.004038,-0.005348,-0.007923,0.011290,0.005809,0.032723,0.024371,0.089321
3,0.053052,-0.002595,1.000000,-0.004559,0.001886,-0.004581,-0.005637,0.001701,-0.003112,-0.005504,...,0.000148,-0.000588,0.011203,-0.004824,0.003674,-0.003255,0.012881,0.008089,-0.002964,0.015953
4,0.176911,-0.003808,-0.004559,1.000000,0.121014,0.065707,0.100595,0.054231,0.002412,0.015607,...,0.072841,0.114280,0.281852,0.039692,0.065483,0.164812,0.115109,0.116843,0.023926,0.062498
5,0.120862,0.013181,0.001886,0.121014,1.000000,0.294134,0.101721,0.426575,-0.004187,0.023468,...,0.061908,0.414929,0.095386,0.254115,0.141073,0.090149,0.145760,0.122600,0.258288,0.040361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.121958,0.011290,-0.003255,0.164812,0.090149,0.047476,0.172484,0.081904,0.057979,0.054858,...,0.153879,0.084190,0.224593,0.035234,0.106729,1.000000,0.115978,0.188312,0.052375,0.093788
607,0.254192,0.005809,0.012881,0.115109,0.145760,0.142158,0.173287,0.178130,0.003252,-0.004817,...,0.080027,0.187581,0.173008,0.126261,0.101129,0.115978,1.000000,0.258232,0.142529,0.098496
608,0.262225,0.032723,0.008089,0.116843,0.122600,0.137932,0.305429,0.175906,0.086221,0.048357,...,0.136304,0.174056,0.164440,0.133722,0.144878,0.188312,0.258232,1.000000,0.109556,0.248902


In [7]:
def get_user_similar_movies( user1, user2 ):
    common_movies = Rating_avg[Rating_avg.userId == user1].merge(
    Rating_avg[Rating_avg.userId == user2],
    on = "movieId",
    how = "inner" )
    return common_movies.merge( movies, on = 'movieId' )

In [8]:
action_lover = [(29,509)]
similar_movies = pd.DataFrame()
                 
for user1,user2 in action_lover:
                 similar_movies = similar_movies.append(get_user_similar_movies(user1,user2),ignore_index=True)
                 
similar_movies = similar_movies.sort_values('rating_x_x',ascending=False)
similar_movies.head(50)

,userId_x,movieId,rating_x_x,timestamp_x,rating_y_x,adg_rating_x,userId_y,rating_x_y,timestamp_y,rating_y_y,adg_rating_y,title,genres
3,29,2028,5.0,1362016835,4.141975,0.858025,509,3.5,1436393168,3.216274,0.283726,Saving Private Ryan (1998),Action|Drama|War
5,29,3578,5.0,1362016881,4.141975,0.858025,509,3.0,1435993248,3.216274,-0.216274,Gladiator (2000),Action|Adventure|Drama
2,29,1198,4.5,1307906966,4.141975,0.358025,509,4.0,1435992293,3.216274,0.783726,Raiders of the Lost Ark (Indiana Jones and the...,Action|Adventure
10,29,99114,4.5,1362016549,4.141975,0.358025,509,4.0,1435993779,3.216274,0.783726,Django Unchained (2012),Action|Drama|Western
12,29,111362,4.5,1405816130,4.141975,0.358025,509,3.5,1435994914,3.216274,0.283726,X-Men: Days of Future Past (2014),Action|Adventure|Sci-Fi
1,29,1136,4.0,1308084085,4.141975,-0.141975,509,4.0,1435993023,3.216274,0.783726,Monty Python and the Holy Grail (1975),Adventure|Comedy|Fantasy
6,29,56174,4.0,1309069586,4.141975,-0.141975,509,3.5,1435993585,3.216274,0.283726,I Am Legend (2007),Action|Horror|Sci-Fi|Thriller|IMAX
7,29,70286,4.0,1307906685,4.141975,-0.141975,509,3.5,1435993497,3.216274,0.283726,District 9 (2009),Mystery|Sci-Fi|Thriller
8,29,81845,4.0,1307907049,4.141975,-0.141975,509,3.5,1435992324,3.216274,0.283726,"King's Speech, The (2010)",Drama
11,29,104841,4.0,1405816277,4.141975,-0.141975,509,2.5,1435994355,3.216274,-0.716274,Gravity (2013),Action|Sci-Fi|IMAX


In [9]:
print(similar_movies['title'])

3                            Saving Private Ryan (1998)
5                                      Gladiator (2000)
2     Raiders of the Lost Ark (Indiana Jones and the...
10                              Django Unchained (2012)
12                    X-Men: Days of Future Past (2014)
1                Monty Python and the Holy Grail (1975)
6                                    I Am Legend (2007)
7                                     District 9 (2009)
8                             King's Speech, The (2010)
11                                       Gravity (2013)
0                                   Pulp Fiction (1994)
4                                Iron Giant, The (1999)
9                             X-Men: First Class (2011)
Name: title, dtype: object


In [10]:
similar_movies

,userId_x,movieId,rating_x_x,timestamp_x,rating_y_x,adg_rating_x,userId_y,rating_x_y,timestamp_y,rating_y_y,adg_rating_y,title,genres
3,29,2028,5.0,1362016835,4.141975,0.858025,509,3.5,1436393168,3.216274,0.283726,Saving Private Ryan (1998),Action|Drama|War
5,29,3578,5.0,1362016881,4.141975,0.858025,509,3.0,1435993248,3.216274,-0.216274,Gladiator (2000),Action|Adventure|Drama
2,29,1198,4.5,1307906966,4.141975,0.358025,509,4.0,1435992293,3.216274,0.783726,Raiders of the Lost Ark (Indiana Jones and the...,Action|Adventure
10,29,99114,4.5,1362016549,4.141975,0.358025,509,4.0,1435993779,3.216274,0.783726,Django Unchained (2012),Action|Drama|Western
12,29,111362,4.5,1405816130,4.141975,0.358025,509,3.5,1435994914,3.216274,0.283726,X-Men: Days of Future Past (2014),Action|Adventure|Sci-Fi
1,29,1136,4.0,1308084085,4.141975,-0.141975,509,4.0,1435993023,3.216274,0.783726,Monty Python and the Holy Grail (1975),Adventure|Comedy|Fantasy
6,29,56174,4.0,1309069586,4.141975,-0.141975,509,3.5,1435993585,3.216274,0.283726,I Am Legend (2007),Action|Horror|Sci-Fi|Thriller|IMAX
7,29,70286,4.0,1307906685,4.141975,-0.141975,509,3.5,1435993497,3.216274,0.283726,District 9 (2009),Mystery|Sci-Fi|Thriller
8,29,81845,4.0,1307907049,4.141975,-0.141975,509,3.5,1435992324,3.216274,0.283726,"King's Speech, The (2010)",Drama
11,29,104841,4.0,1405816277,4.141975,-0.141975,509,2.5,1435994355,3.216274,-0.716274,Gravity (2013),Action|Sci-Fi|IMAX


In [11]:
similar_movies.describe()

,userId_x,movieId,rating_x_x,timestamp_x,rating_y_x,adg_rating_x,userId_y,rating_x_y,timestamp_y,rating_y_y,adg_rating_y
count,13.0,13.000000,13.000000,1.300000e+01,13.000000,13.000000,13.0,13.000000,1.300000e+01,1.300000e+01,13.000000
mean,29.0,47834.692308,4.153846,1.339722e+09,4.141975,0.011871,509.0,3.538462,1.436035e+09,3.216274e+00,0.322187
std,0.0,46459.374100,0.515777,3.843472e+07,0.000000,0.515777,0.0,0.431158,1.118137e+05,4.622227e-16,0.431158
min,29.0,296.000000,3.500000,1.307906e+09,4.141975,-0.641975,509.0,2.500000,1.435992e+09,3.216274e+00,-0.716274
25%,29.0,2028.000000,4.000000,1.307907e+09,4.141975,-0.141975,509.0,3.500000,1.435993e+09,3.216274e+00,0.283726
50%,29.0,56174.000000,4.000000,1.309070e+09,4.141975,-0.141975,509.0,3.500000,1.435994e+09,3.216274e+00,0.283726
75%,29.0,87232.000000,4.500000,1.362017e+09,4.141975,0.358025,509.0,4.000000,1.435995e+09,3.216274e+00,0.783726
max,29.0,111362.000000,5.000000,1.405816e+09,4.141975,0.858025,509.0,4.000000,1.436393e+09,3.216274e+00,0.783726


In [12]:
similar_movies.shape

(13, 13)